In [1]:
from nets.yolo import YoloBody
import torch
import collections

## SwinYOLO=709
## backbone=617, head=108
## backbone.backbone=361
### backbone.backbone.patch_embed=4,
### backbone.backbone.layers=357
#### layers.0=28,layers.1=31,layers.2=255,layers.3=31,layers.4=12(即spp)
## bu_conv2=6,bu_conv1=6, lateral=6, reduce_conv=6, c3=216   记为other：240

## other(240) + head(108) + layers.4(12)= 360 from YOLOX
## backbone.backbone.patch_embed(4) + backbone.backbone.layers(0-3)(345)=349 from swin

## 查看model其中的参数（非体系）

In [2]:
model = YoloBody(num_classes=80, phi="l")
weight = model.state_dict()
i = 0
for param_tensor in weight: # 字典的遍历默认是遍历 key，所以param_tensor实际上是键值
    print(param_tensor,'\t',weight[param_tensor].size())
    print(len(weight), i)

backbone.backbone.patch_embed.proj.weight 	 torch.Size([128, 3, 4, 4])
709 0
backbone.backbone.patch_embed.proj.bias 	 torch.Size([128])
709 0
backbone.backbone.patch_embed.norm.weight 	 torch.Size([128])
709 0
backbone.backbone.patch_embed.norm.bias 	 torch.Size([128])
709 0
backbone.backbone.layers.0.blocks.0.norm1.weight 	 torch.Size([128])
709 0
backbone.backbone.layers.0.blocks.0.norm1.bias 	 torch.Size([128])
709 0
backbone.backbone.layers.0.blocks.0.attn.relative_position_bias_table 	 torch.Size([529, 4])
709 0
backbone.backbone.layers.0.blocks.0.attn.relative_position_index 	 torch.Size([144, 144])
709 0
backbone.backbone.layers.0.blocks.0.attn.qkv.weight 	 torch.Size([384, 128])
709 0
backbone.backbone.layers.0.blocks.0.attn.qkv.bias 	 torch.Size([384])
709 0
backbone.backbone.layers.0.blocks.0.attn.proj.weight 	 torch.Size([128, 128])
709 0
backbone.backbone.layers.0.blocks.0.attn.proj.bias 	 torch.Size([128])
709 0
backbone.backbone.layers.0.blocks.0.norm2.weight 	 torch.Siz

backbone.C3_p3.m.0.conv2.conv.weight 	 torch.Size([128, 128, 3, 3])
709 0
backbone.C3_p3.m.0.conv2.bn.weight 	 torch.Size([128])
709 0
backbone.C3_p3.m.0.conv2.bn.bias 	 torch.Size([128])
709 0
backbone.C3_p3.m.0.conv2.bn.running_mean 	 torch.Size([128])
709 0
backbone.C3_p3.m.0.conv2.bn.running_var 	 torch.Size([128])
709 0
backbone.C3_p3.m.0.conv2.bn.num_batches_tracked 	 torch.Size([])
709 0
backbone.C3_p3.m.1.conv1.conv.weight 	 torch.Size([128, 128, 1, 1])
709 0
backbone.C3_p3.m.1.conv1.bn.weight 	 torch.Size([128])
709 0
backbone.C3_p3.m.1.conv1.bn.bias 	 torch.Size([128])
709 0
backbone.C3_p3.m.1.conv1.bn.running_mean 	 torch.Size([128])
709 0
backbone.C3_p3.m.1.conv1.bn.running_var 	 torch.Size([128])
709 0
backbone.C3_p3.m.1.conv1.bn.num_batches_tracked 	 torch.Size([])
709 0
backbone.C3_p3.m.1.conv2.conv.weight 	 torch.Size([128, 128, 3, 3])
709 0
backbone.C3_p3.m.1.conv2.bn.weight 	 torch.Size([128])
709 0
backbone.C3_p3.m.1.conv2.bn.bias 	 torch.Size([128])
709 0
backbone.C

## 原来YOLOX=750
## backbone=642,head = 108
## backbone.backbone = 402(CSPDarknet)
## bu_conv2=6,bu_conv1=6, lateral=6, reduce_conv=6, c3=216   记为other：240
## backbone.backnone分别是stem， dark2， dark3， dark4， dark5.
## spp在dark5中，占12个,为backbone.backbone.dark5.1. 。。。。
## 我们需要的是 other(240) + head(108) + spp(12) = 360

In [3]:
path = r"G:\学习资料\目标检测\yolox_l.pth"



no_backbone_weight = torch.load(path)
# print(type(no_backbone_weight))
i = 0
# for k in list(no_backbone_weight.keys()):
#     if "backbone.backbone.dark" in k:
#         i += 1
#         del no_backbone_weight[k]
#     if "backbone" in k:
#         i += 1
#         del no_backbone_weight[k]
for name, para in no_backbone_weight.items():
    print(name,"   ", para.size())
    print(len(no_backbone_weight))
print(i)

backbone.backbone.stem.conv.conv.weight     torch.Size([64, 12, 3, 3])
750
backbone.backbone.stem.conv.bn.weight     torch.Size([64])
750
backbone.backbone.stem.conv.bn.bias     torch.Size([64])
750
backbone.backbone.stem.conv.bn.running_mean     torch.Size([64])
750
backbone.backbone.stem.conv.bn.running_var     torch.Size([64])
750
backbone.backbone.stem.conv.bn.num_batches_tracked     torch.Size([])
750
backbone.backbone.dark2.0.conv.weight     torch.Size([128, 64, 3, 3])
750
backbone.backbone.dark2.0.bn.weight     torch.Size([128])
750
backbone.backbone.dark2.0.bn.bias     torch.Size([128])
750
backbone.backbone.dark2.0.bn.running_mean     torch.Size([128])
750
backbone.backbone.dark2.0.bn.running_var     torch.Size([128])
750
backbone.backbone.dark2.0.bn.num_batches_tracked     torch.Size([])
750
backbone.backbone.dark2.1.conv1.conv.weight     torch.Size([64, 128, 1, 1])
750
backbone.backbone.dark2.1.conv1.bn.weight     torch.Size([64])
750
backbone.backbone.dark2.1.conv1.bn.bias 

head.cls_convs.0.1.bn.running_mean     torch.Size([256])
750
head.cls_convs.0.1.bn.running_var     torch.Size([256])
750
head.cls_convs.0.1.bn.num_batches_tracked     torch.Size([])
750
head.cls_convs.1.0.conv.weight     torch.Size([256, 256, 3, 3])
750
head.cls_convs.1.0.bn.weight     torch.Size([256])
750
head.cls_convs.1.0.bn.bias     torch.Size([256])
750
head.cls_convs.1.0.bn.running_mean     torch.Size([256])
750
head.cls_convs.1.0.bn.running_var     torch.Size([256])
750
head.cls_convs.1.0.bn.num_batches_tracked     torch.Size([])
750
head.cls_convs.1.1.conv.weight     torch.Size([256, 256, 3, 3])
750
head.cls_convs.1.1.bn.weight     torch.Size([256])
750
head.cls_convs.1.1.bn.bias     torch.Size([256])
750
head.cls_convs.1.1.bn.running_mean     torch.Size([256])
750
head.cls_convs.1.1.bn.running_var     torch.Size([256])
750
head.cls_convs.1.1.bn.num_batches_tracked     torch.Size([])
750
head.cls_convs.2.0.conv.weight     torch.Size([256, 256, 3, 3])
750
head.cls_convs.2.0.bn.

## SwinTransformer = 364
## stem=4, head=2, norm=2
## layers=356
## layers.0 = 32,layers.1=32, layers.2=264, layers.3 = 28,

In [4]:
path = r"G:\学习资料\目标检测\swin_base_patch4_window12_384.pth"
backbone_weight = torch.load(path)
for name, para in backbone_weight.items():
    backbone_weight = para
i = 0
for name, para in backbone_weight.items():
    print(name, "     ", para.size())
    print(len(backbone_weight), i)
#     i += 1
# for k in list(backbone_weight.keys()):
#     if "layers.2" in k:
#         i += 1
# print(i)

patch_embed.proj.weight       torch.Size([128, 3, 4, 4])
364 0
patch_embed.proj.bias       torch.Size([128])
364 0
patch_embed.norm.weight       torch.Size([128])
364 0
patch_embed.norm.bias       torch.Size([128])
364 0
layers.0.blocks.0.norm1.weight       torch.Size([128])
364 0
layers.0.blocks.0.norm1.bias       torch.Size([128])
364 0
layers.0.blocks.0.attn.qkv.weight       torch.Size([384, 128])
364 0
layers.0.blocks.0.attn.qkv.bias       torch.Size([384])
364 0
layers.0.blocks.0.attn.proj.weight       torch.Size([128, 128])
364 0
layers.0.blocks.0.attn.proj.bias       torch.Size([128])
364 0
layers.0.blocks.0.norm2.weight       torch.Size([128])
364 0
layers.0.blocks.0.norm2.bias       torch.Size([128])
364 0
layers.0.blocks.0.mlp.fc1.weight       torch.Size([512, 128])
364 0
layers.0.blocks.0.mlp.fc1.bias       torch.Size([512])
364 0
layers.0.blocks.0.mlp.fc2.weight       torch.Size([128, 512])
364 0
layers.0.blocks.0.mlp.fc2.bias       torch.Size([128])
364 0
layers.0.blocks.1

## 获得backbone.backbone.patch_embed(4)

In [5]:
path = r"G:\学习资料\目标检测\swin_base_patch4_window12_384.pth"
backbone_weight = torch.load(path)
for name, para in backbone_weight.items():
    backbone_weight = para
i = 0

temp1 = collections.OrderedDict()
for k in list(backbone_weight.keys()):
    if "patch_embed" in k:
        new_name = "backbone.backbone." + k
        temp1.update({new_name: backbone_weight[k]})
#         i += 1
#         del no_backbone_weight[k]
# print(i)

patch_embed.proj.weight       torch.Size([128, 3, 4, 4])
364 0
patch_embed.proj.bias       torch.Size([128])
364 0
patch_embed.norm.weight       torch.Size([128])
364 0
patch_embed.norm.bias       torch.Size([128])
364 0
layers.0.blocks.0.norm1.weight       torch.Size([128])
364 0
layers.0.blocks.0.norm1.bias       torch.Size([128])
364 0
layers.0.blocks.0.attn.qkv.weight       torch.Size([384, 128])
364 0
layers.0.blocks.0.attn.qkv.bias       torch.Size([384])
364 0
layers.0.blocks.0.attn.proj.weight       torch.Size([128, 128])
364 0
layers.0.blocks.0.attn.proj.bias       torch.Size([128])
364 0
layers.0.blocks.0.norm2.weight       torch.Size([128])
364 0
layers.0.blocks.0.norm2.bias       torch.Size([128])
364 0
layers.0.blocks.0.mlp.fc1.weight       torch.Size([512, 128])
364 0
layers.0.blocks.0.mlp.fc1.bias       torch.Size([512])
364 0
layers.0.blocks.0.mlp.fc2.weight       torch.Size([128, 512])
364 0
layers.0.blocks.0.mlp.fc2.bias       torch.Size([128])
364 0
layers.0.blocks.1

In [6]:
# for k, v in temp6.items():
#     print(k,  v.size())

## 获得backbone.backbone.layers（0-3）：349

## layers.0(28 = 32-3(downsample）-1(attn_mask))  (我们保留了attn)

In [7]:
path = r"G:\学习资料\目标检测\swin_base_patch4_window12_384.pth"
backbone_weight = torch.load(path)
for name, para in backbone_weight.items():
    backbone_weight = para

i = 0
temp2 = collections.OrderedDict()
for k in list(backbone_weight.keys()):
    if "layers.0.block" in k:
        new_name = "backbone.backbone." + k
#         print(k, backbone_weight[k].size())
        temp2.update({new_name:backbone_weight[k]})
        i += 1

print(i)


29


## layers.1(31=28+3)(保留了attn_mask，实际为32)

In [8]:
path = r"G:\学习资料\目标检测\swin_base_patch4_window12_384.pth"
backbone_weight = torch.load(path)
for name, para in backbone_weight.items():
    backbone_weight = para

i = 0
temp3 = collections.OrderedDict()
for k in list(backbone_weight.keys()):
    if "layers.0.downsample" in k:
        p = k.replace("layers.0", "layers.1")
        new_name = "backbone.backbone." + p
        temp3.update({new_name:backbone_weight[k]})
    if "layers.1.blocks" in k:
        new_name = "backbone.backbone." + k
        temp3.update({new_name:backbone_weight[k]})
#         i += 1
#         new_name = "backbone.backbone." + k
#         print(k, backbone_weight[k].size())
#         temp2.update({new_name:backbone_weight[k]})
#         i += 1

# print(i)


## layers.2(255=3+14 * 18) （保留了attn_mask，实际为264）

In [9]:
path = r"G:\学习资料\目标检测\swin_base_patch4_window12_384.pth"
backbone_weight = torch.load(path)
for name, para in backbone_weight.items():
    backbone_weight = para

i = 0
temp4 = collections.OrderedDict()
for k in list(backbone_weight.keys()):
    if "layers.1.downsample" in k:
        p = k.replace("layers.1", "layers.2")
        new_name = "backbone.backbone." + p
        temp4.update({new_name:backbone_weight[k]})
        i += 1
    if "layers.2.blocks" in k:
        new_name = "backbone.backbone." + k
        temp4.update({new_name:backbone_weight[k]})
        i += 1
#         new_name = "backbone.backbone." + k
#         print(k, backbone_weight[k].size())
#         temp2.update({new_name:backbone_weight[k]})
#         i += 1

print(i)


264


In [10]:
for k in list(temp4.keys()):
    print(k, temp4[k].size())

backbone.backbone.layers.2.downsample.norm.weight torch.Size([1024])
backbone.backbone.layers.2.downsample.norm.bias torch.Size([1024])
backbone.backbone.layers.2.blocks.0.norm1.weight torch.Size([512])
backbone.backbone.layers.2.blocks.0.norm1.bias torch.Size([512])
backbone.backbone.layers.2.blocks.0.attn.qkv.weight torch.Size([1536, 512])
backbone.backbone.layers.2.blocks.0.attn.qkv.bias torch.Size([1536])
backbone.backbone.layers.2.blocks.0.attn.proj.weight torch.Size([512, 512])
backbone.backbone.layers.2.blocks.0.attn.proj.bias torch.Size([512])
backbone.backbone.layers.2.blocks.0.norm2.weight torch.Size([512])
backbone.backbone.layers.2.blocks.0.norm2.bias torch.Size([512])
backbone.backbone.layers.2.blocks.0.mlp.fc1.weight torch.Size([2048, 512])
backbone.backbone.layers.2.blocks.0.mlp.fc1.bias torch.Size([2048])
backbone.backbone.layers.2.blocks.0.mlp.fc2.weight torch.Size([512, 2048])
backbone.backbone.layers.2.blocks.0.mlp.fc2.bias torch.Size([512])
backbone.backbone.layers.

## layers.3(31= 2 * 14 +3) （从layers.3开始没有attn_mask）

In [11]:
path = r"G:\学习资料\目标检测\swin_base_patch4_window12_384.pth"
backbone_weight = torch.load(path)
for name, para in backbone_weight.items():
    backbone_weight = para

i = 0
temp5 = collections.OrderedDict()
for k in list(backbone_weight.keys()):
    if "layers.2.downsample" in k:
        p = k.replace("layers.2", "layers.3")
        new_name = "backbone.backbone." + p
        temp5.update({new_name:backbone_weight[k]})
        i += 1
    if "layers.3.blocks" in k:
        new_name = "backbone.backbone." + k
        temp5.update({new_name:backbone_weight[k]})
        i += 1
#         new_name = "backbone.backbone." + k
#         print(k, backbone_weight[k].size())
#         temp2.update({new_name:backbone_weight[k]})
#         i += 1

print(i)


31


## layers.4(12,即spp)

In [12]:
path = r"G:\学习资料\目标检测\yolox_l.pth"
no_backbone_weight = torch.load(path)
i = 0

temp6 = collections.OrderedDict()
for k in list(no_backbone_weight.keys()):
    if "backbone.backbone.dark5.1" in k:
#         i += 1
        new_name = k.replace("dark5.1", "layers.4")
        temp6.update({new_name:no_backbone_weight[k]})
#         new_name = "backbone.backbone." + k
#         temp1.update({new_name: backbone_weight[k]})
#         i += 1
#         del no_backbone_weight[k]
print(i)

0


## others(240)

In [13]:
path = r"G:\学习资料\目标检测\yolox_l.pth"
no_backbone_weight = torch.load(path)

i = 0

temp7 = collections.OrderedDict()
for k in list(no_backbone_weight.keys()):
    if "backbone.backbone" in k:
        del no_backbone_weight[k]
#         i += 1
#         new_name = k.replace("dark5.1", "layers.4")
#         temp6.update({new_name:no_backbone_weight[k]})
for k in list(no_backbone_weight.keys()):
    if "backbone" in k:
#         i += 1
        temp7.update({k:no_backbone_weight[k]})
print(i)

0


In [14]:
len(temp7)

240

## head(108)

In [15]:
path = r"G:\学习资料\目标检测\yolox_l.pth"
no_backbone_weight = torch.load(path)

i = 0

temp8 = collections.OrderedDict()

#         i += 1
#         new_name = k.replace("dark5.1", "layers.4")
#         temp6.update({new_name:no_backbone_weight[k]})
for k in list(no_backbone_weight.keys()):
    if "head" in k:
#         i += 1
        temp8.update({k:no_backbone_weight[k]})
print(i)

0


In [16]:
for i,k in temp8.items():
    print(i, k.size())

head.cls_convs.0.0.conv.weight torch.Size([256, 256, 3, 3])
head.cls_convs.0.0.bn.weight torch.Size([256])
head.cls_convs.0.0.bn.bias torch.Size([256])
head.cls_convs.0.0.bn.running_mean torch.Size([256])
head.cls_convs.0.0.bn.running_var torch.Size([256])
head.cls_convs.0.0.bn.num_batches_tracked torch.Size([])
head.cls_convs.0.1.conv.weight torch.Size([256, 256, 3, 3])
head.cls_convs.0.1.bn.weight torch.Size([256])
head.cls_convs.0.1.bn.bias torch.Size([256])
head.cls_convs.0.1.bn.running_mean torch.Size([256])
head.cls_convs.0.1.bn.running_var torch.Size([256])
head.cls_convs.0.1.bn.num_batches_tracked torch.Size([])
head.cls_convs.1.0.conv.weight torch.Size([256, 256, 3, 3])
head.cls_convs.1.0.bn.weight torch.Size([256])
head.cls_convs.1.0.bn.bias torch.Size([256])
head.cls_convs.1.0.bn.running_mean torch.Size([256])
head.cls_convs.1.0.bn.running_var torch.Size([256])
head.cls_convs.1.0.bn.num_batches_tracked torch.Size([])
head.cls_convs.1.1.conv.weight torch.Size([256, 256, 3, 3]

## 汇总所有的权值（709+11（attn_mask）=720）

In [17]:
temp = collections.OrderedDict()
temp.update(temp1)
temp.update(temp2)
temp.update(temp3)
temp.update(temp4)
temp.update(temp5)
temp.update(temp6)
temp.update(temp7)
temp.update(temp8)
print(len(temp))

720


In [21]:
torch.save(temp, "G:/学习资料/目标检测/swinyolo.pth")

In [24]:
path = r"G:\学习资料\目标检测\swinyolo.pth"
true_weight = torch.load(path)

In [25]:
for k in list(true_weight.keys()):
    if "attn_mask" in k:
#         i += 1
#         temp8.update({k:no_backbone_weight[k]})
        del true_weight[k]
new_weight = true_weight
print(len(new_weight))

709


In [26]:
torch.save(temp, "G:/学习资料/目标检测/true_swinyolo.pth")

In [ ]:
model = YoloBody(num_classes=80, phi="l")
model.load_state_dict(true_weight, strict=False)

## 查看具体某一层的详细信息，注意其与参数是有变化的

In [18]:
model = YoloBody(num_classes=80, phi="l")
# 查看模型及参数
# print(model.backbone.backbone.state_dict())

# 查看具体某一层的详细信息，注意其与参数是有变化的
# print(model.backbone.backbone.patch_embed)
print(type(model.backbone.backbone.patch_embed.proj.bias)) # nn.parameters
print(type(model.backbone.backbone.patch_embed.proj.bias.data)) # tensor
print(type(model.backbone.backbone.patch_embed.proj.bias.grad)) # 还没计算就是None

<class 'torch.nn.parameter.Parameter'>
<class 'torch.Tensor'>
<class 'NoneType'>


In [19]:
# 下划线在函数名后面表示是替换函数，会直接替换掉变量原来的值
# net.add_module('name', block())

In [20]:
print(model)

YoloBody(
  (backbone): YOLOPAFPN(
    (backbone): SwinTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (layers): ModuleList(
        (0): BasicLayer(
          (blocks): ModuleList(
            (0): SwinTransformerBlock(
              (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attn): WindowAttention(
                (qkv): Linear(in_features=128, out_features=384, bias=True)
                (attn_drop): Dropout(p=0.0, inplace=False)
                (proj): Linear(in_features=128, out_features=128, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (softmax): Softmax(dim=-1)
              )
              (drop_path): Identity()
              (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (mlp): M